In [1]:
from __future__ import division
import pandas as pd
import numpy as np
from TumorTruths import TumorTruths
from scipy.stats import mode
from sklearn.metrics import f1_score
from sklearn import cluster
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances

In [2]:
svnCalls = pd.read_csv('SNVCalls_IS4.txt', delimiter='\t', dtype={'CHROM':pd.np.str})
truths = TumorTruths('synthetic.challenge.set4.tumour.25pctmasked.truth.vcf')

In [3]:
resultsArray = truths.GetTruths(svnCalls)

In [4]:
mostCommonPredictions = mode(svnCalls[svnCalls.columns[3:-1]], axis=1)[0]
f1_score(mostCommonPredictions, resultsArray)

0.83674707935062975

In [7]:
num_clusters = 5
clusters, centers = kClustering(svnCalls[svnCalls.columns[3:-1]].T.values, np.median, manhattan_distances, num_clusters)
# k_means = cluster.KMeans(n_clusters=num_clusters)
# k_means.fit(svnCalls[svnCalls.columns[3:-1]].T)

In [8]:
def TransformEuclideanPointsToPredictions(point):
    return 1 if point >= 0.50 else 0

toPredictions = np.vectorize(TransformEuclideanPointsToPredictions) 
clusterPredictions = np.zeros(len(resultsArray))
for i in range(0, num_clusters):
#     weight = len([1 for num in k_means.labels_  if num == i ])
    weight = len([1 for num in clusters  if num == i ])
#     distances = manhattan_distances(svnCalls[svnCalls.columns[3:-1]].T, k_means.cluster_centers_[i])
    distances = manhattan_distances(svnCalls[svnCalls.columns[3:-1]].T, centers[i])
    bestIndex = np.argmin(distances)
    representativeCaller = svnCalls[svnCalls.columns[3 + bestIndex]]
    clusterPredictions += weight * representativeCaller

# clusterPredictions /= len(k_means.labels_)
clusterPredictions /= len(clusters)
clusterPredictions = toPredictions(clusterPredictions)

In [9]:
f1_score(clusterPredictions, resultsArray)

0.87254795315276124

In [ ]:
from __future__ import division
import numpy as np
import math
from sklearn.metrics import pairwise

def kMedoids(data, func, dist_func, k=5):

    cluster = np.zeros(len(data))
    finished = False
    medoids = np.copy(data[0:5])

    for i in range(len(data)):
        distances = dist_func(data[i], medoids)
        cluster[i] = np.argmin(distances)
        
    for i in range(k):
        
            
#             d1 = dist_func(data[i], center1)
#             d2 = dist_func(data[i], center2)
#             newVal = 0 if (min(d1, d2) == d1) else 1

#             if (newVal != cluster[i]):
#                 finished = False
# 
#             cluster[i] = newVal
#         center1 = func(data[np.where(cluster == 0)], axis=0)
#         center2 = func(data[np.where(cluster == 1)], axis=0)

#     return cluster, centers

In [ ]:
kMedoids(svnCalls[svnCalls.columns[3:-1]].T.values, np.median, manhattan_distances)

In [ ]:
import pam

results = pam.kmedoids(svnCalls[svnCalls.columns[3:-1]].T, 5)

In [ ]:
print results[2]

In [ ]:
def TransformEuclideanPointsToPredictions(point):
    return 1 if point >= 0.5 else 0

toPredictions = np.vectorize(TransformEuclideanPointsToPredictions) 
clusterPredictions = np.zeros(len(resultsArray))
for i in range(0, 5):
    weight = len(results[2][results[1][i]])
    print weight
    representativeCaller = svnCalls[svnCalls.columns[3 + results[1][i]]] 
#     print representativeCaller
    clusterPredictions += weight * representativeCaller

print np.max(clusterPredictions)
print len(clusterPredictions)
clusterPredictions /= len(svnCalls.columns) - 3
clusterPredictions = toPredictions(clusterPredictions)

In [ ]:
print np.max(clusterPredictions)

In [ ]:
print cosine_similarity(svnCalls[svnCalls.columns[58]], svnCalls[svnCalls.columns[6]] )

In [6]:
def kClustering(data, func, dist_func, k):

    clusters = np.zeros(len(data))
    centers = np.copy(data[0:k])
    
    nextPossibleStartingCenter = 1
    centersChosen = 1
    while centersChosen < k:
        centerAlreadyIncluded = False
        
        for i in range(centersChosen):
            centerAlreadyIncluded |= np.array_equal(centers[i], data[nextPossibleStartingCenter])
                
        if not centerAlreadyIncluded:
            centers[centersChosen] = data[nextPossibleStartingCenter]
            centersChosen += 1
            
        nextPossibleStartingCenter += 1

    while True:
        distances = dist_func(data, centers)
        newClusters = np.argmin(distances, axis=1)
    
        if np.array_equal(newClusters, clusters):
            return clusters, centers

        clusters = newClusters
        centers = np.array([func(data[np.where(clusters == i)], axis=0) for i in range(k)])

data = np.array(((0, 0),(0, 1),(0, 2),(0, 3),(0, 4),(0, 5),(1, 0),(1, 1),(1, 2),(1, 3),(1, 4),(1, 5),(3, 4),(3, 5)))

In [ ]:
print (svnCalls[svnCalls.columns[3:-1]]).T.values[1].shape